In [0]:
pip install openpyxl

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, MapType, DateType,IntegerType
from pyspark.sql import functions as f
import json
from pyspark.sql.window import Window


#Raw data


In [0]:
raw_dataframe = spark.read.format("csv").option("header", True).load("dbfs:/FileStore/tables/project_data-2.csv")
display(raw_dataframe)

member_id,first_name,middle_name,last_name,address,city,state,ZIP,phones,email,gender,date_of_birth,year_of_birth
1,Patricia,Micah,Lancaster,688-1952 Sociis Avenue,Fort Worth,Tennessee,1211963,8045196412,pede.nec@aol.ca,male,3/11/1967,1967
2,Liberty,Macaulay,Shepard,Ap #742-9426 Libero Av.,Columbia,Vermont,83762,5783456321,rutrum.magna.cras@outlook.,female,3/3/1963,1963
3,Courtney,Connor,Tyler,263-7625 Purus. Ave,Broken Arrow,Hawaii,99838,6254417402,est.nunc.laoreet@hotmail.com,other,3/17/1969,1969
4,Simon,Jamal,Stanley,"P.O. Box 337, 8613 Ullamcorper Rd.",Augusta,Wisconsin,56822,8534665522,ipsum.donec@google.com,female,6/6/1974,1974
5,Wynne,Oleg,Hickman,7463 Gravida Ave,Toledo,Vermont,88836,7036159685,nulla.facilisis@google.edu,female,12/2/1938,1938
6,Orson,Raymond,Benton,617-2915 Faucibus Street,Baltimore,Oklahoma,154146,1375438046,ante.vivamus@outlook.couk,null,4/6/1958,1958
7,Ashely,Keefe,Keith,Ap #696-3298 Lacus. St.,Rutland,Indiana,67366,7937671657,mattis@icloud.com,female,9/1/1937,1937
8,Kirestin,Elton,Ellison,Ap #961-2005 Duis Road,Joliet,Vermont,95915,9826767215,convallis.dolor.quisque@hotmail.com,male,10/25/1978,1978
9,Price,Anthony,Downs,315-6044 Quisque St.,Vancouver,Hawaii,74723,9612481367,class.aptent.taciti@protonmail.edu,female,11/5/1923,1923
10,Buckminster,Rahim,Webb,Ap #980-5073 Gravida Avenue,Reno,Connecticut,75277,4648651487,quam@icloud.org,other,11/29/2001,2001


# Transformed dataframe

In [0]:
transformed_dataframe = spark.read.format("csv").option("header", True).load("dbfs:/FileStore/tables/project_data-2.csv")\
    .withColumn("date_of_birth", f.when(
        f.to_date(f.col("date_of_birth"),"M/d/yyyy") < f.current_date(),
        f.to_date(f.col("date_of_birth"),"M/d/yyyy")
    ))\
    .withColumn("ZIP", f.when(
        f.length(f.col('ZIP'))==5,
        f.col('ZIP')
        ).otherwise(
            f.when(
                f.length(f.col('ZIP'))>5,
                 f.substring(f.col('ZIP'), 1, 5)
            )
        ))\
    .withColumn("phones", f.when(
        f.length(f.col('phones'))==10,
        f.col('phones')
    ).otherwise(
        f.when(
            f.length(f.col("phones"))>10,
            f.substring(f.col("phones"),1,10)
        )
    ))\
    .withColumn("email",f.when(
        f.col("email").rlike(r'^[\w]+[\w\.-]*@[\w\.-]+\.\w+$'),
        f.col("email")
    ))\
    .withColumn("year_of_birth", f.when(
        f.col('date_of_birth') < f.current_date(),
        f.col('year_of_birth')
    ))\
    .withColumn("address_details",f.concat_ws(', ','address','City','State',f.when(
        f.length(f.col('ZIP'))==5,
        f.col('ZIP')
        ).otherwise(
            f.when(
                f.length(f.col('ZIP'))>5,
                 f.substring(f.col('ZIP'), 1, 5)
            )
        )))


transformed_dataframe = transformed_dataframe.drop_duplicates()
display(transformed_dataframe)

member_id,first_name,middle_name,last_name,address,city,state,ZIP,phones,email,gender,date_of_birth,year_of_birth,address_details
65,Upton,Jonas,Mckinney,126-9841 Tincidunt Avenue,Boise,Oregon,85831,6952462924,diam.vel@icloud.ca,female,1984-01-27,1984,"126-9841 Tincidunt Avenue, Boise, Oregon, 85831"
131,Lionel,Emery,Hale,717-1536 Dis Rd.,Essex,Nebraska,36252,0886189269,donec.est@icloud.couk,other,1951-06-21,1951,"717-1536 Dis Rd., Essex, Nebraska, 36252"
196,Christine,Garrett,Kelley,328-8604 Ipsum Rd.,Reading,Kansas,47760,9147337609,ante.blandit.viverra@aol.couk,null,1971-08-19,1971,"328-8604 Ipsum Rd., Reading, Kansas, 47760"
294,Merritt,Chester,Shannon,875 Dui. St.,Lowell,Florida,73476,8061145636,ligula.tortor.dictum@hotmail.couk,female,2009-10-01,2009,"875 Dui. St., Lowell, Florida, 73476"
515,Armand,Carter,Vaughn,Ap #455-9086 Nec Av.,Honolulu,Texas,56926,3264986272,aliquam.adipiscing@aol.net,male,1963-01-22,1963,"Ap #455-9086 Nec Av., Honolulu, Texas, 56926"
951,Timon,Lamar,Keith,336-7048 Mauris Street,Covington,Nebraska,72256,4271219182,pellentesque.eget@google.com,male,1948-07-29,1948,"336-7048 Mauris Street, Covington, Nebraska, 72256"
63,null,Tucker,Marks,"P.O. Box 348, 9565 Adipiscing Rd.",Saint Paul,Illinois,47533,9340592443,amet@protonmail.edu,other,1947-06-30,1947,"P.O. Box 348, 9565 Adipiscing Rd., Saint Paul, Illinois, 47533"
351,Eric,Yoshio,Crawford,"713-4648 At, Rd.",Lincoln,Washington,52871,7466842360,orci.luctus@hotmail.edu,female,2004-02-25,2004,"713-4648 At, Rd., Lincoln, Washington, 52871"
557,Jason,Calvin,Franks,"P.O. Box 354, 7248 Rutrum St.",Springdale,Colorado,74791,5762430714,integer.mollis@protonmail.net,female,1946-10-08,1946,"P.O. Box 354, 7248 Rutrum St., Springdale, Colorado, 74791"
565,Mufutau,Isaac,Watkins,"P.O. Box 925, 5571 Curabitur Ave",Knoxville,Hawaii,56572,7841224816,venenatis.vel@yahoo.ca,male,1940-08-05,1940,"P.O. Box 925, 5571 Curabitur Ave, Knoxville, Hawaii, 56572"


#Data Profiling function

In [0]:
def data_profiling(raw_dataframe, transformed_dataframe):

    total_columns_dist = raw_dataframe.columns
    for i in transformed_dataframe.columns:
        if i not in total_columns_dist:
            total_columns_dist.append(i)


    diff_distinct_record = {}
    diff_null_count = {}
    diff_not_null_records = {}
    diff_per_null_records = {}

    for col in total_columns_dist:
        if col in raw_dataframe.columns:
            raw_distinct_count = raw_dataframe.select(f.countDistinct(col)).collect()[0][0]
            raw_null_count = raw_dataframe.filter(f.col(col).isNull()).count()
            raw_not_null_count = raw_dataframe.filter(f.col(col).isNotNull()).count()
            
            diff_distinct_record[col] = {"raw": raw_distinct_count}
            diff_null_count[col] = {"raw": raw_null_count}
            diff_not_null_records[col] = {"raw": format(raw_distinct_count/raw_not_null_count*100, ".2f") + "%"}
            diff_per_null_records[col] = {"raw": format(raw_null_count/raw_not_null_count*100, ".2f") + "%"}
            
            if col not in transformed_dataframe.columns:
                diff_distinct_record[col]['target'] = "not present"
                diff_null_count[col]['target'] =  "not present"
                diff_not_null_records[col]['target'] =  "not present"
                diff_per_null_records[col]['target'] =  "not present"
            
            if col in transformed_dataframe.columns:
                target_distinct_count = transformed_dataframe.select(f.countDistinct(col)).collect()[0][0]
                target_null_count = transformed_dataframe.filter(f.col(col).isNull()).count()
                target_null_not_null_count = transformed_dataframe.filter(f.col(col).isNotNull()).count()
                   
                diff_distinct_record[col]['target'] = target_distinct_count
                diff_null_count[col]['target'] = target_null_count
                diff_not_null_records[col]['target'] = format(target_distinct_count/target_null_not_null_count*100, ".2f") + "%"
                diff_per_null_records[col]['target'] = format(target_null_count/target_null_not_null_count*100, ".2f") + "%"
                

        else:
            target_distinct_count = transformed_dataframe.select(f.countDistinct(col)).collect()[0][0]
            target_null_count = transformed_dataframe.filter(f.col(col).isNull()).count()
            target_null_not_null_count = transformed_dataframe.filter(f.col(col).isNotNull()).count()
            
            if col not in raw_dataframe.columns:
                diff_distinct_record[col] = {"raw": "not present"}
                diff_null_count[col] = {"raw": "not present"}
                diff_not_null_records[col] = {"raw": "not present"}
                diff_per_null_records[col] = {"raw": "not present"}
            diff_distinct_record[col]['target'] = target_distinct_count
            diff_null_count[col]['target'] = target_null_count
            diff_not_null_records[col]['target'] = format(target_distinct_count/target_null_not_null_count*100, ".2f") + "%"
            diff_per_null_records[col]['target'] = format(target_null_count/target_null_not_null_count*100, ".2f") + "%"
            

    import pandas as pd

    #Validate Row Count
    print("Validating Row Count :")

    # Calculate raw and target row counts
    raw_row_count = raw_dataframe.count()
    target_row_count = transformed_dataframe.count()

    # Check if row counts match
    result = "MATCH" if raw_row_count == target_row_count else "MISMATCH"

    # Create a DataFrame
    data = {
        "Raw_Row_Count": [raw_row_count],
        "Target_Row_Count": [target_row_count],
        "Result": [result]
    }
    row_count_df = pd.DataFrame(data)
    display(row_count_df)

    # Create DataFrames for each validation
    distinct_record_counts = pd.DataFrame(diff_distinct_record).T.reset_index().rename(columns={"index": "Columns"})
    Null_record_counts = pd.DataFrame(diff_null_count).T.reset_index().rename(columns={"index": "Columns"})
    distinct_record_percenatge = pd.DataFrame(diff_not_null_records).T.reset_index().rename(columns={"index": "Columns"})
    Null_record_percenatge = pd.DataFrame(diff_per_null_records).T.reset_index().rename(columns={"index": "Columns"})

    # Add result column to DataFrames
    distinct_record_counts['Result'] = distinct_record_counts.apply(lambda row: "Match" if row['raw'] == row['target'] else "Mismatch", axis=1)
    Null_record_counts['Result'] = Null_record_counts.apply(lambda row: "Match" if row['raw'] == row['target'] else f"Mismatch", axis=1)
    distinct_record_percenatge['Result'] = distinct_record_percenatge.apply(lambda row: "Match" if row['raw'] == row['target'] else "Mismatch", axis=1)
    Null_record_percenatge['Result'] = Null_record_percenatge.apply(lambda row: "Match" if row['raw'] == row['target'] else "Mismatch", axis=1)

    print("distinct_record_counts :")
    display(distinct_record_counts)
    print("Null_record_counts :")
    display(Null_record_counts)
    print("distinct_record_percenatge :")
    display(distinct_record_percenatge)
    print("Null_record_percenatge :")
    display(Null_record_percenatge)


    #Saving the output into an excel file

    with pd.ExcelWriter("/tmp/data_profile_output.xlsx") as writer:
        row_count_df.to_excel(writer, sheet_name="row_count_df", index=False)
        distinct_record_counts.to_excel(writer, sheet_name="distinct_record_counts", index=False)
        Null_record_counts.to_excel(writer, sheet_name="Null_record_counts", index=False)
        distinct_record_percenatge.to_excel(writer, sheet_name="distinct_record_percenatge", index=False)
        Null_record_percenatge.to_excel(writer, sheet_name="Null_record_percenatge", index=False)

    dbutils.fs.mv('file:/tmp/data_profile_output.xlsx','dbfs:/FileStore/project/data_profiling/data_profile_output.xlsx')

In [0]:
data_profiling(raw_dataframe, transformed_dataframe)

Validating Row Count :


Raw_Row_Count,Target_Row_Count,Result
1004,1000,MISMATCH


distinct_record_counts :


/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Could not convert 'not present' with type str: tried to convert to int64
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


Columns,raw,target,Result
member_id,1000,1000,Match
first_name,677,677,Match
middle_name,440,440,Match
last_name,635,635,Match
address,1000,1000,Match
city,204,204,Match
state,40,40,Match
ZIP,993,990,Mismatch
phones,1000,997,Mismatch
email,994,984,Mismatch


...

Columns,raw,target,Result
member_id,0,0,Match
first_name,3,3,Match
middle_name,6,6,Match
last_name,4,4,Match
address,0,0,Match
city,0,0,Match
state,0,0,Match
ZIP,0,3,Mismatch
phones,0,3,Mismatch
email,0,10,Mismatch


distinct_record_percenatge :


Columns,raw,target,Result
member_id,99.60%,100.00%,Mismatch
first_name,67.63%,67.90%,Mismatch
middle_name,44.09%,44.27%,Mismatch
last_name,63.50%,63.76%,Mismatch
address,99.60%,100.00%,Mismatch
city,20.32%,20.40%,Mismatch
state,3.98%,4.00%,Mismatch
ZIP,98.90%,99.30%,Mismatch
phones,99.60%,100.00%,Mismatch
email,99.00%,99.39%,Mismatch


Null_record_percenatge :


Columns,raw,target,Result
member_id,0.00%,0.00%,Match
first_name,0.30%,0.30%,Match
middle_name,0.60%,0.60%,Match
last_name,0.40%,0.40%,Match
address,0.00%,0.00%,Match
city,0.00%,0.00%,Match
state,0.00%,0.00%,Match
ZIP,0.00%,0.30%,Mismatch
phones,0.00%,0.30%,Mismatch
email,0.00%,1.01%,Mismatch


#DDL Validation function

In [0]:
def validate_ddl(raw_dataframe,transformed_dataframe):
    # Validating the column count
    print("Validate column count :")

    raw_column_count = len(raw_dataframe.columns)
    target_column_count = len(transformed_dataframe.columns)
    result = "MATCH" if raw_column_count == target_column_count else "MISMATCH"

    schema = StructType([
    StructField("Raw_Columns", StringType(), True),
    StructField("Target_Columns", StringType(), True),
    StructField("Result", StringType(), True)
])
    data = [(raw_column_count, target_column_count, result)]
    column_count = spark.createDataFrame(data, schema)
    column_count.show()

    #Validating the Common columns
    schema = StructType([
    StructField("Raw_Columns", StringType(), True),
    ])
    raw_col = spark.createDataFrame([(col,) for col in raw_dataframe.columns], ["Raw_Columns"])
    raw_col = raw_col.withColumn("Index", f.row_number().over(Window.orderBy(f.monotonically_increasing_id())))
    target_col = spark.createDataFrame([(col,) for col in transformed_dataframe.columns], ["Target_Columns"])
    target_col = target_col.withColumn("Index", f.row_number().over(Window.orderBy(f.monotonically_increasing_id())))

    all_col = raw_col.join(target_col,"Index",how = 'outer')
    all_col = all_col.withColumn("Result",f.when(f.col("Raw_Columns")==f.col("Target_Columns"),"Match").otherwise("Mismatch"))

    print("Common Columns in both dataframes :")
    common_col = all_col.select(f.col("Raw_Columns").alias("Common_Columns")).filter(f.col("Result")=="Match")
    common_col.show()

    print("Mismatched Columns in both dataframes :")
    mismatched_col = all_col.select(f.col("Raw_Columns"),f.col("Target_Columns")).filter(f.col("Result")=="Mismatch")
    mismatched_col.show()

     # Validate column data types

    raw_data_types = dict(raw_dataframe.dtypes)
    target_data_types = dict(transformed_dataframe.dtypes)
    total_columns = {}
    total_columns.update(target_data_types)
    total_columns.update(raw_data_types)
    total_datatypes = {}
    mismatched_datatypes = {}
    for key in total_columns:
        total_datatypes[key] = {"raw": raw_data_types.get(key, 'Not present'), "target": target_data_types.get(key, 'Not present')}
        if total_datatypes[key]["raw"] != total_datatypes[key]["target"]:
            mismatched_datatypes[key] = total_datatypes[key]

    data = [(key, value['raw'], value['target']) for key, value in mismatched_datatypes.items()]

    schema = StructType([
        StructField("Column", StringType(), True),
        StructField("Raw", StringType(), True),
        StructField("Target", StringType(), True)
    ])

    data_type_df = spark.createDataFrame(data, schema)
    print("Validating Mismatched Datatypes :")
    data_type_df.show()

    #Validate Row Count
    print("Validating Row Count :")

    raw_row_count = raw_dataframe.count()
    target_row_count = transformed_dataframe.count()
    result = "MATCH" if raw_row_count == target_row_count else "MISMATCH"

    schema = StructType([
    StructField("Raw_Columns", StringType(), True),
    StructField("Target_Columns", StringType(), True),
    StructField("Result", StringType(), True)
])
    data = [(raw_row_count, target_row_count, result)]
    row_count = spark.createDataFrame(data, schema)
    row_count.show()

    #Saving the output into an excel file
    import pandas as pd
    with pd.ExcelWriter("/tmp/ddl_output.xlsx") as writer:
        column_count.toPandas().to_excel(writer, sheet_name="column_count", index=False)
        common_col.toPandas().to_excel(writer, sheet_name="common_col", index=False)
        mismatched_col.toPandas().to_excel(writer, sheet_name="mismatched_col", index=False)
        data_type_df.toPandas().to_excel(writer, sheet_name="data_type_df", index=False)
        row_count.toPandas().to_excel(writer, sheet_name="row_count", index=False)

    dbutils.fs.mv('file:/tmp/ddl_output.xlsx','dbfs:/FileStore/project/ddl/ddl_output.xlsx')






In [0]:
validate_ddl(raw_dataframe,transformed_dataframe)

Validate column count :
+-----------+--------------+--------+
|Raw_Columns|Target_Columns|  Result|
+-----------+--------------+--------+
|         13|            14|MISMATCH|
+-----------+--------------+--------+

Common Columns in both dataframes :
+--------------+
|Common_Columns|
+--------------+
|     member_id|
|    first_name|
|   middle_name|
|     last_name|
|       address|
|          city|
|         state|
|           ZIP|
|        phones|
|         email|
|        gender|
| date_of_birth|
| year_of_birth|
+--------------+

Mismatched Columns in both dataframes :
+-----------+---------------+
|Raw_Columns| Target_Columns|
+-----------+---------------+
|       null|address_details|
+-----------+---------------+

Validating Mismatched Datatypes :
+---------------+-----------+------+
|         Column|        Raw|Target|
+---------------+-----------+------+
|  date_of_birth|     string|  date|
|address_details|Not present|string|
+---------------+-----------+------+

Validating 

#function to download excel sheet

In [0]:
def download_output(link):
    ddl_link_prefix = "https://community.cloud.databricks.com/files/project/ddl/ddl_output.xlsx"
    data_profiling_link_prefix = "https://community.cloud.databricks.com/files/project/data_profiling/data_profile_output.xlsx"
    link_split = link.split("#")
    link_1 = link_split[0]
    notebook_id = link_1.split(".com/")
    ddl_op_link = ddl_link_prefix +notebook_id[1]
    data_profile_op_link = data_profiling_link_prefix +notebook_id[1]
    print("ddl :",ddl_op_link)
    print("data_profiling",data_profile_op_link)


In [0]:
download_output(link="https://community.cloud.databricks.com/?o=3902034117996389#notebook/394176186215233/command/1762007057422543")

ddl : https://community.cloud.databricks.com/files/project/ddl/ddl_output.xlsx?o=3902034117996389
data_profiling https://community.cloud.databricks.com/files/project/data_profiling/data_profile_output.xlsx?o=3902034117996389
